Let's check are we in collab or not! Also set, are we in ultratraining mode or not.

In [2]:
import sys
IN_COLAB = 'google.colab' in sys.modules


IS_TRAIN = False
EPOCH_COUNT = 2


First of all let's connect our drive to save logs properly. It differs between collab environment and non-colab

In [3]:
import pathlib

if IN_COLAB:
    from google.colab import drive
    BASE_PATH = pathlib.Path("/content/drive")
    DRIVE_PATH = BASE_PATH / "MyDrive"
    drive.mount(str(BASE_PATH), force_remount=True)
    
    SAVE_PATH = DRIVE_PATH / "gdl-usi-2024/"
    SAVE_PATH.mkdir(exist_ok=True)
else:
    BASE_PATH = pathlib.Path("experiment_data/")
    SAVE_PATH = BASE_PATH
    

Let's download our toolset for the preparation of our data. Also we need to install some special tooling for our arrangements.

In [4]:
if IN_COLAB:
    !wget https://raw.githubusercontent.com/strategy155/gdl-usi-2024-spring/dev/GDL_data_prep.ipynb --output-document GDL_data_prep.ipynb
    !pip install ipynb
else: 
    pass

Then let's load our data preparation module, and the pandas also. Then we import pandas, download the data, and read it in a usual way.

In [5]:
from ipynb.fs.defs.GDL_data_prep import get_data, compute_diff_ts, minmax_bound_scaler, check_nans
import pandas as pd

DATASET_NAME = "merged_raw_dataset.parquet"

# get_data() # downloading the data to the environment, NOT WORKNIG TEMPORALILY
stocknet_df = pd.read_parquet(DATASET_NAME)
stocknet_df.head()

XOM                                                        \
                 Open       High        Low      Close  Adj Close    Volume   
Date                                                                          
2012-09-04  87.330002  87.459999  86.519997  87.120003  74.408150  10602900   
2012-09-05  87.519997  87.650002  87.059998  87.330002  74.587517  10113100   
2012-09-06  87.889999  89.250000  87.849998  88.989998  76.005302  15713500   
2012-09-07  89.269997  90.000000  89.269997  89.919998  76.799614  15623900   
2012-09-10  89.820000  90.000000  89.400002  89.480003  76.423813  10750700   

                  AEP                                   ...        NGG  \
                 Open       High        Low      Close  ...        Low   
Date                                                    ...              
2012-09-04  43.000000  43.250000  42.830002  43.139999  ...  59.388645   
2012-09-05  43.349998  43.349998  42.919998  42.950001  ...  59.868996   
2012-09-06  43.150002  43.580002  43.029999  43.520000  ...  60.414848   
2012-09-07  43.549999  43.669998  43.369999  43.459999  ...  60.316593   
2012-09-10  43.689999  43.689999  43.380001  43.389999  ...  59.454147   

                                                 UN                        \
                Close  Adj Close  Volume       Open       High        Low   
Date                                                                        
2012-09-04  59.847160  47.161697  239300  34.990002  35.029999  34.799999   
2012-09-05  60.163757  47.411182  283100  35.180000  35.180000  34.869999   
2012-09-06  60.796944  47.910160  337200  35.200001  35.740002  35.200001   
2012-09-07  60.556770  47.720898  204700  35.619999  35.700001  35.480000   
2012-09-10  59.454147  46.851994  390300  34.880001  35.070000  34.880001   

                                           
                Close  Adj Close   Volume  
Date                                       
2012-09-04  34.910000  29.580452  1138300  
2012-09-05  34.930000  29.597395  1213200  
2012-09-06  35.669998  30.224426  2269400  
2012-09-07  35.520000  30.097328  2945600  
2012-09-10  34.950001  29.614346  2224500  

[5 rows x 528 columns]

Then we compute differences of our tsdata, then apply the scaler.

In [6]:
stocknet_diff, invert_minmax = minmax_bound_scaler(
                    compute_diff_ts(stocknet_df)
                )
stocknet_diff.head()

XOM                                                    \
                Open      High       Low     Close Adj Close    Volume   
Date                                                                     
2012-09-05  0.230977  0.200000  0.229211  0.210244  0.207508  0.129380   
2012-09-06  0.234719  0.238108  0.234542  0.245610  0.240483  0.157003   
2012-09-07  0.255717  0.215135  0.247974  0.227805  0.223882  0.156561   
2012-09-10  0.238462  0.194865  0.220469  0.194390  0.192726  0.132525   
2012-09-11  0.225572  0.193513  0.220896  0.208536  0.205916  0.125613   

                 AEP                                ...       NGG            \
                Open      High       Low     Close  ...       Low     Close   
Date                                                ...                       
2012-09-05  0.209677  0.218681  0.215676  0.225657  ...  0.230242  0.228490   
2012-09-06  0.189964  0.224396  0.216216  0.256364  ...  0.231700  0.235053   
2012-09-07  0.211469  0.218242  0.222432  0.230909  ...  0.217360  0.216949   
2012-09-10  0.202151  0.215165  0.213514  0.230505  ...  0.200346  0.199071   
2012-09-11  0.191039  0.208132  0.213243  0.234950  ...  0.214200  0.224190   

                                      UN                                \
           Adj Close    Volume      Open      High       Low     Close   
Date                                                                     
2012-09-05  0.231741  0.113655  0.173067  0.153626  0.171162  0.173156   
2012-09-06  0.237243  0.117469  0.168533  0.162637  0.178623  0.187316   
2012-09-07  0.222064  0.108128  0.179200  0.149451  0.177188  0.169813   
2012-09-10  0.207074  0.121212  0.148267  0.136484  0.151937  0.161554   
2012-09-11  0.228135  0.130327  0.176800  0.160440  0.177188  0.181023   

                                
           Adj Close    Volume  
Date                            
2012-09-05  0.173101  0.104209  
2012-09-06  0.185273  0.109228  
2012-09-07  0.170227  0.112441  
2012-09-10  0.163127  0.109014  
2012-09-11  0.179863  0.114450  

[5 rows x 528 columns]

Finally we check the data for the gaps.

In [7]:
check_nans(stocknet_diff)

The nans are distributed over 6 stocks, affecting 961 rows and 42 columns, therefore it is advised to drop the columns


Then we decided to remove the poorly formed columns. It concludes the first step of our data preparation.

In [10]:
stocknet_diff.dropna(axis=1,inplace=True)
stocknet_diff.head()

XOM                                                    \
                Open      High       Low     Close Adj Close    Volume   
Date                                                                     
2012-09-05  0.230977  0.200000  0.229211  0.210244  0.207508  0.129380   
2012-09-06  0.234719  0.238108  0.234542  0.245610  0.240483  0.157003   
2012-09-07  0.255717  0.215135  0.247974  0.227805  0.223882  0.156561   
2012-09-10  0.238462  0.194865  0.220469  0.194390  0.192726  0.132525   
2012-09-11  0.225572  0.193513  0.220896  0.208536  0.205916  0.125613   

                 AEP                                ...       NGG            \
                Open      High       Low     Close  ...       Low     Close   
Date                                                ...                       
2012-09-05  0.209677  0.218681  0.215676  0.225657  ...  0.230242  0.228490   
2012-09-06  0.189964  0.224396  0.216216  0.256364  ...  0.231700  0.235053   
2012-09-07  0.211469  0.218242  0.222432  0.230909  ...  0.217360  0.216949   
2012-09-10  0.202151  0.215165  0.213514  0.230505  ...  0.200346  0.199071   
2012-09-11  0.191039  0.208132  0.213243  0.234950  ...  0.214200  0.224190   

                                      UN                                \
           Adj Close    Volume      Open      High       Low     Close   
Date                                                                     
2012-09-05  0.231741  0.113655  0.173067  0.153626  0.171162  0.173156   
2012-09-06  0.237243  0.117469  0.168533  0.162637  0.178623  0.187316   
2012-09-07  0.222064  0.108128  0.179200  0.149451  0.177188  0.169813   
2012-09-10  0.207074  0.121212  0.148267  0.136484  0.151937  0.161554   
2012-09-11  0.228135  0.130327  0.176800  0.160440  0.177188  0.181023   

                                
           Adj Close    Volume  
Date                            
2012-09-05  0.173101  0.104209  
2012-09-06  0.185273  0.109228  
2012-09-07  0.170227  0.112441  
2012-09-10  0.163127  0.109014  
2012-09-11  0.179863  0.114450  

[5 rows x 486 columns]

Let's create our baseline graph, based on the additional information. Let's load our table!

In [8]:
if IN_COLAB:
    !wget https://raw.githubusercontent.com/yumoxu/stocknet-dataset/master/StockTable --output-document "stocktable.tsv"
    STOCKTABLES_PATH = pathlib.Path("stocktable.tsv")
else:
    STOCKTABLES_PATH = pathlib.Path("stocknet-dataset/StockTable")
    
stock_sectors_df = pd.read_table(STOCKTABLES_PATH)
stock_sectors_df.head()

,Sector,Symbol,Company
0,Basic Matierials,$XOM,Exxon Mobil Corporation
1,Basic Matierials,$RDS-B,Royal Dutch Shell
2,Basic Matierials,$PTR,PetroChina Company Limited
3,Basic Matierials,$CVX,Chevron Corporation
4,Basic Matierials,$TOT,TOTAL S.A.


Now let's build an adjacency matrix based on that information (everything inside a sector is interconnected). First let's remove a dollar sign from our symbol names in the table, to use the exact order of symbols we have in our stocknet_diff dataset.

In [9]:
stock_sectors_df["Symbol"] =stock_sectors_df["Symbol"].str.lstrip("$")
stock_sectors_df["Symbol"].head()

0      XOM
1    RDS-B
2      PTR
3      CVX
4      TOT
Name: Symbol, dtype: object

We normalize the order of the columns to be the same, so our adjacency will have the exact order of things.

In [10]:
stock_sectors_df = stock_sectors_df.set_index("Symbol")
stocknet_diff_column_order = stocknet_diff.columns.get_level_values(0).unique()
stock_sectors_df = stock_sectors_df.reindex(stocknet_diff_column_order)
stock_sectors_df.head()

,Sector,Company
XOM,Basic Matierials,Exxon Mobil Corporation
AEP,Utilities,American Electric Power Inc.
JNJ,Healthcare,Johnson & Johnson
BHP,Basic Matierials,BHP Billiton Limited
BCH,Financial,Banco de Chile


Let's create row-by-row our adjacency matrix, based on the type of the sector of the company, by iterating throught the symbol names of the stocknet_diff.

In [11]:
import numpy as np

adjacency_rows = []
for stockname_to_sector in stock_sectors_df.itertuples():
  is_sector_equal_row = stockname_to_sector.Sector == stock_sectors_df["Sector"]
  adjacency_rows.append(is_sector_equal_row.astype(int).values)

adjacency_matrix_by_sector = np.stack(adjacency_rows)
adjacency_matrix_by_sector

array([[1, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 1, 0],
       [0, 0, 1, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 1, 0, 1],
       [0, 1, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 1, 0, 1]])

Let's install the tsl, and the dependencies

In [12]:
# Install required packages.
import os
import torch
if IN_COLAB:
    os.environ['TORCH'] = torch.__version__
    print(torch.__version__)
    
    !pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
    !pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
    !pip install -q git+https://github.com/pyg-team/pytorch_geometric.git
    !pip install -q git+https://github.com/TorchSpatiotemporal/tsl.git


We also need to convert the adjacency matrix to the edge_index format for PyG compatibility.

In [13]:
from tsl.ops.connectivity import adj_to_edge_index

edge_index_sector = adj_to_edge_index(adjacency_matrix_by_sector)
edge_index_sector

(array([[ 0,  0,  0, ..., 80, 80, 80],
        [ 0,  3, 24, ..., 65, 78, 80]]),
 array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 

Let's create a torch-based dataset for further training.

In [14]:
from tsl.data import SpatioTemporalDataset

tsl_stock_df = SpatioTemporalDataset(target=stocknet_diff,
                                     connectivity=edge_index_sector,
                                      horizon=5,
                                      window=12,
                                      stride=1)
print(tsl_stock_df)

SpatioTemporalDataset(n_samples=1241, n_nodes=81, n_channels=6)


Let's checkup the sample

In [15]:
sample = tsl_stock_df[0]
print(sample)

Data(
  input=(x=[t=12, n=81, f=6], edge_index=[2, e=751], edge_weight=[e=751]),
  target=(y=[t=5, n=81, f=6]),
  has_mask=False
)


Let's see the pattern of our data:

In [16]:
sample.pattern

{'x': 't n f', 'edge_index': '2 e', 'edge_weight': 'e', 'y': 't n f'}

Let's split our data sequentially. Also, let's setup it in a good way, and perform all the computations!

In [17]:
from tsl.data.datamodule import (SpatioTemporalDataModule,
                                 TemporalSplitter)
import numpy as np

BATCH_SIZE = 64
VAL_LEN = 0.1
TEST_LEN = 0.1

stocknet_splitter = TemporalSplitter(val_len=VAL_LEN, test_len=TEST_LEN)
stocknet_datamodule = SpatioTemporalDataModule(
    dataset=tsl_stock_df,
    splitter=stocknet_splitter,
    batch_size=BATCH_SIZE,
)

stocknet_datamodule.setup()
print(stocknet_datamodule)

SpatioTemporalDataModule(train_len=994, val_len=99, test_len=124, scalers=[], batch_size=64)


Let's try a simple TTSM model.

In [18]:
import torch.nn as nn

from tsl.nn.blocks.encoders import RNN
from tsl.nn.layers import NodeEmbedding, DiffConv
from einops.layers.torch import Rearrange  # reshape data with Einstein notation


class TimeThenSpaceModel(nn.Module):
    def __init__(self, input_size: int, n_nodes: int, horizon: int,
                 hidden_size: int = 32,
                 rnn_layers: int = 1,
                 gnn_kernel: int = 2):
        super(TimeThenSpaceModel, self).__init__()

        self.encoder = nn.Linear(input_size, hidden_size)

        self.node_embeddings = NodeEmbedding(n_nodes, hidden_size)

        self.time_nn = RNN(input_size=hidden_size,
                           hidden_size=hidden_size,
                           n_layers=rnn_layers,
                           cell='gru',
                           return_only_last_state=True)

        self.space_nn = DiffConv(in_channels=hidden_size,
                                 out_channels=hidden_size,
                                 k=gnn_kernel)

        self.decoder = nn.Linear(hidden_size, input_size * horizon)
        self.rearrange = Rearrange('b n (t f) -> b t n f', t=horizon)

    def forward(self, x, edge_index, edge_weight):
        x_enc = self.encoder(x)  
        x_emb = x_enc + self.node_embeddings() 
        h = self.time_nn(x_emb)  
        z = self.space_nn(h, edge_index, edge_weight)  
        x_out = self.decoder(z) 
        x_horizon = self.rearrange(x_out)
        return x_horizon

Let's actually train our model now!

In [19]:
hidden_size = 16   #@param
rnn_layers = 1     #@param
gnn_kernel = 2     #@param

input_size = tsl_stock_df.n_channels   # 6 channel
n_nodes = tsl_stock_df.n_nodes         # 81 nodes
horizon = tsl_stock_df.horizon         # 5 time steps

stocknet_ttsm = TimeThenSpaceModel(input_size=input_size,
                           n_nodes=n_nodes,
                           horizon=horizon,
                           hidden_size=hidden_size,
                           rnn_layers=rnn_layers,
                           gnn_kernel=gnn_kernel)
print(stocknet_ttsm)

TimeThenSpaceModel(
  (encoder): Linear(in_features=6, out_features=16, bias=True)
  (node_embeddings): NodeEmbedding(n_nodes=81, embedding_size=16)
  (time_nn): RNN(
    (rnn): GRU(16, 16)
  )
  (space_nn): DiffConv(16, 16)
  (decoder): Linear(in_features=16, out_features=30, bias=True)
  (rearrange): Rearrange('b n (t f) -> b t n f', t=5)
)


Let's setup the predictor then.

In [20]:
from tsl.metrics.torch import MaskedMAE, MaskedMAPE
from tsl.engines import Predictor

stocknet_loss_fn = MaskedMAE()

stocknet_metrics = {'mae': MaskedMAE(),
           'mape': MaskedMAPE(),
           }

stocknet_ttsm_predictor = Predictor(
    model=stocknet_ttsm,           
    optim_class=torch.optim.Adam,  
    optim_kwargs={'lr': 0.001},    
    loss_fn=stocknet_loss_fn,      
    metrics=stocknet_metrics       
)

Let's setup our logging facility!

In [21]:
from pytorch_lightning.loggers import TensorBoardLogger

MODEL_NAME = "stocknet-sector-window-12-forecast-5"
VERSION = 0
MODEL_PATH = SAVE_PATH / MODEL_NAME
MODEL_PATH.mkdir(exist_ok=True)
stocknet_logger = TensorBoardLogger(save_dir=SAVE_PATH, name=MODEL_NAME, version=VERSION)

Launching tensorboard then!

In [11]:
%load_ext tensorboard
%tensorboard --logdir $SAVE_PATH


Let's use a lightning trainer for our dirty purposes.

In [23]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint

stocknet_checkpoint_callback = ModelCheckpoint(
    dirpath=MODEL_PATH,
    save_top_k=1,
    monitor='val_mae',
    mode='min',
    save_last=True
)

stocknet_ttsm_trainer = pl.Trainer(max_epochs=5,
                     logger=stocknet_logger,
                     limit_train_batches=2000,  # end an epoch after 100 updates
                     callbacks=[stocknet_checkpoint_callback])
checkpoint_paths = [checkpoint for checkpoint in MODEL_PATH.iterdir() if checkpoint.suffix == ".ckpt"]
current_checkpoint_used = checkpoint_paths[0] if checkpoint_paths else None
stocknet_ttsm_trainer.fit(stocknet_ttsm_predictor, datamodule=stocknet_datamodule, ckpt_path=current_checkpoint_used)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/parf/.cache/pypoetry/virtualenvs/gnn-usi-24-tWyMBYAO-py3.11/lib/python3.11/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:653: Checkpoint directory /home/parf/projects/gnn-usi-24/experiment_data/stocknet-sector-window-12-forecast-5 exists and is not empty.
Restoring states from the checkpoint path at experiment_data/stocknet-sector-window-12-forecast-5/epoch=231-step=3480.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type               | Params
-----------------------------------------------------
0 | loss_fn       | MaskedMAE          | 0     
1 | train_metrics | MetricCollection   | 0     
2 | val_metrics   | MetricCollection   | 0     
3 | test_metrics  | MetricCollection   | 0     
4 | model         | TimeThenSpaceModel | 4.8 K 
-----------------------------------------------------
4.8 K   

Sanity Checking: |                                                                                                                          | 0/? [00:00<?, ?it/s]

/home/parf/.cache/pypoetry/virtualenvs/gnn-usi-24-tWyMBYAO-py3.11/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Sanity Checking DataLoader 0:   0%|                                                                                                         | 0/2 [00:00<?, ?it/s]

Only args ['edge_weight', 'x', 'edge_index'] are forwarded to the model (TimeThenSpaceModel). 


/home/parf/.cache/pypoetry/virtualenvs/gnn-usi-24-tWyMBYAO-py3.11/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.
/home/parf/.cache/pypoetry/virtualenvs/gnn-usi-24-tWyMBYAO-py3.11/lib/python3.11/site-packages/pytorch_lightning/loops/fit_loop.py:298: The number of training batches (15) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Epoch 232: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:00<00:00, 31.02it/s, v_num=0]
Validation: |                                                                                                                               | 0/? [00:00<?, ?it/s]
Epoch 233: 100%|████████████████████████████████████| 15/15 [00:00<00:00, 47.38it/s, v_num=0, val_mae=0.012, val_mape=0.0645, train_mae=0.0116, train_mape=0.0616]
Validation: |                                                                                                                               | 0/? [00:00<?, ?it/s]
Epoch 234: 100%|████████████████████████████████████| 15/15 [00:00<00:00, 42.74it/s, v_num=0, val_mae=0.012, val_mape=0.0644, train_mae=0.0116, train_mape=0.0617]
Validation: |                                                                                                                               | 0/? [00:00<?, ?it/s]
Epoch 235: 100%|██████

`Trainer.fit` stopped: `max_epochs=240` reached.


Epoch 239: 100%|█████████████████████████████████████| 15/15 [00:00<00:00, 39.21it/s, v_num=0, val_mae=0.012, val_mape=0.064, train_mae=0.0116, train_mape=0.0616]


That's nearly perfect. Let's see the test accuracy here.

In [24]:
stocknet_ttsm_predictor.load_model(stocknet_checkpoint_callback.best_model_path)
stocknet_ttsm_predictor.freeze()

stocknet_ttsm_trainer.test(stocknet_ttsm_predictor, datamodule=stocknet_datamodule);

Predictor with already instantiated model is loading a state_dict from /home/parf/projects/gnn-usi-24/experiment_data/stocknet-sector-window-12-forecast-5/epoch=231-step=3480.ckpt. Cannot  check if model hyperparameters are the same.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/parf/.cache/pypoetry/virtualenvs/gnn-usi-24-tWyMBYAO-py3.11/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 48.77it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_loss           0.01033993810415268
        test_mae            0.01033993810415268
        test_mape           0.05546041578054428
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


Let's put all of this to a function, which will let us to go through a grid of hyperparams. I'd like to use hyperopt, or raytune, but not now I guess.

In [25]:
PARAM_COMBINATIONS = [(10, 1, 16, 2, 2), 
                     (100, 3, 32, 4,4)]

def train_stocknet_sector_horizon(window_size, stride, hidden_size, rnn_layers, gnn_kernel):
    tsl_stock_df = SpatioTemporalDataset(target=stocknet_diff,
                                     connectivity=edge_index_sector,
                                      horizon=5,
                                      window=window_size,
                                      stride=stride)
    stocknet_splitter = TemporalSplitter(val_len=VAL_LEN, test_len=TEST_LEN)

    stocknet_datamodule = SpatioTemporalDataModule(
        dataset=tsl_stock_df,
        splitter=stocknet_splitter,
        batch_size=BATCH_SIZE,
    )

    stocknet_datamodule.setup()

    
    input_size = tsl_stock_df.n_channels   # 6 channel
    n_nodes = tsl_stock_df.n_nodes         # 81 nodes
    horizon = tsl_stock_df.horizon         # 5 time steps
    
    stocknet_ttsm = TimeThenSpaceModel(input_size=input_size,
                               n_nodes=n_nodes,
                               horizon=horizon,
                               hidden_size=hidden_size,
                               rnn_layers=rnn_layers,
                               gnn_kernel=gnn_kernel)

    stocknet_loss_fn = MaskedMAE()

    stocknet_metrics = {'mae': MaskedMAE(),
               'mape': MaskedMAPE(),
               }
    
    # setup predictor
    stocknet_ttsm_predictor = Predictor(
        model=stocknet_ttsm,                   # our initialized model
        optim_class=torch.optim.Adam,  # specify optimizer to be used...
        optim_kwargs={'lr': 0.001},    # ...and parameters for its initialization
        loss_fn=stocknet_loss_fn,               # which loss function to be used
        metrics=stocknet_metrics                # metrics to be logged during train/val/test
    )

    MODEL_NAME = f"stocknet-sector-w-{window_size}-s-{stride}-h-{hidden_size}-rl-{rnn_layers}-gk-{gnn_kernel}"
    VERSION = 0
    MODEL_PATH = SAVE_PATH / MODEL_NAME
    MODEL_PATH.mkdir(exist_ok=True)
    stocknet_logger = TensorBoardLogger(save_dir=SAVE_PATH, name=MODEL_NAME, version=VERSION)

    stocknet_checkpoint_callback = ModelCheckpoint(
        dirpath=MODEL_PATH,
        save_top_k=1,
        monitor='val_mae',
        mode='min',
        save_last=True

    )
    
    stocknet_ttsm_trainer = pl.Trainer(max_epochs=EPOCH_COUNT,
                         logger=stocknet_logger,
                         limit_train_batches=2000,  # end an epoch after 100 updates
                         callbacks=[stocknet_checkpoint_callback])
    checkpoint_paths = [checkpoint for checkpoint in MODEL_PATH.iterdir() if checkpoint.suffix == ".ckpt"]
    current_checkpoint_used = checkpoint_paths[0] if checkpoint_paths else None
    stocknet_ttsm_trainer.fit(stocknet_ttsm_predictor, datamodule=stocknet_datamodule, ckpt_path=current_checkpoint_used)

    
if IS_TRAIN:
    for parameters_combination in PARAM_COMBINATIONS:
        train_stocknet_sector_horizon(*parameters_combination)

I've been checking our data, to see if it is finely looking. It looks a bit strangely, so we'll make different scaling strategy later.

In [26]:
sample = tsl_stock_df[0]
print(sample.y[:,0,:])
stocknet_diff.head(17)

tensor([[0.2663, 0.2095, 0.2507, 0.2149, 0.2118, 0.1762],
        [0.2121, 0.1943, 0.2104, 0.2041, 0.2018, 0.1339],
        [0.2437, 0.2046, 0.2228, 0.2017, 0.1995, 0.1487],
        [0.2189, 0.1805, 0.2083, 0.1927, 0.1911, 0.1368],
        [0.2293, 0.2005, 0.2196, 0.2190, 0.2157, 0.1319]])


XOM                                                    \
                Open      High       Low     Close Adj Close    Volume   
Date                                                                     
2012-09-05  0.230977  0.200000  0.229211  0.210244  0.207508  0.129380   
2012-09-06  0.234719  0.238108  0.234542  0.245610  0.240483  0.157003   
2012-09-07  0.255717  0.215135  0.247974  0.227805  0.223882  0.156561   
2012-09-10  0.238462  0.194865  0.220469  0.194390  0.192726  0.132525   
2012-09-11  0.225572  0.193513  0.220896  0.208536  0.205916  0.125613   
2012-09-12  0.224532  0.193514  0.214712  0.203415  0.201140  0.128738   
2012-09-13  0.228690  0.239730  0.218124  0.246097  0.240938  0.177509   
2012-09-14  0.264241  0.217568  0.256503  0.231219  0.227066  0.184996   
2012-09-17  0.235759  0.197567  0.224094  0.195610  0.193863  0.138162   
2012-09-18  0.216840  0.187297  0.214499  0.198780  0.196820  0.133508   
2012-09-19  0.228067  0.182162  0.200000  0.178780  0.178171  0.159906   
2012-09-20  0.199168  0.193243  0.207463  0.228292  0.224336  0.139863   
2012-09-21  0.266320  0.209459  0.250746  0.214878  0.211829  0.176168   
2012-09-24  0.212058  0.194324  0.210448  0.204146  0.201822  0.133871   
2012-09-25  0.243659  0.204595  0.222814  0.201707  0.199549  0.148728   
2012-09-26  0.218919  0.180541  0.208315  0.192683  0.191134  0.136795   
2012-09-27  0.229314  0.200540  0.219616  0.219024  0.215695  0.131943   

                 AEP                                ...       NGG            \
                Open      High       Low     Close  ...       Low     Close   
Date                                                ...                       
2012-09-05  0.209677  0.218681  0.215676  0.225657  ...  0.230242  0.228490   
2012-09-06  0.189964  0.224396  0.216216  0.256364  ...  0.231700  0.235053   
2012-09-07  0.211469  0.218242  0.222432  0.230909  ...  0.217360  0.216949   
2012-09-10  0.202151  0.215165  0.213514  0.230505  ...  0.200346  0.199071   
2012-09-11  0.191039  0.208132  0.213243  0.234950  ...  0.214200  0.224190   
2012-09-12  0.195341  0.214725  0.209730  0.236162  ...  0.224652  0.223738   
2012-09-13  0.191756  0.231868  0.215135  0.251919  ...  0.231214  0.241615   
2012-09-14  0.216846  0.212088  0.217838  0.221212  ...  0.229999  0.222833   
2012-09-17  0.191039  0.211209  0.212162  0.229697  ...  0.216145  0.216949   
2012-09-18  0.189606  0.202418  0.206757  0.231313  ...  0.227325  0.228943   
2012-09-19  0.199283  0.229231  0.218108  0.245859  ...  0.224652  0.226906   
2012-09-20  0.203226  0.223956  0.221351  0.238990  ...  0.222221  0.233016   
2012-09-21  0.209677  0.212967  0.215676  0.234949  ...  0.212985  0.210160   
2012-09-24  0.198925  0.234506  0.220000  0.251111  ...  0.217603  0.225774   
2012-09-25  0.210036  0.226593  0.221622  0.229697  ...  0.229270  0.223511   
2012-09-26  0.196057  0.212967  0.211351  0.233333  ...  0.213471  0.217175   
2012-09-27  0.196416  0.198022  0.198919  0.211515  ...  0.229513  0.231658   

                                      UN                                \
           Adj Close    Volume      Open      High       Low     Close   
Date                                                                     
2012-09-05  0.231741  0.113655  0.173067  0.153626  0.171162  0.173156   
2012-09-06  0.237243  0.117469  0.168533  0.162637  0.178623  0.187316   
2012-09-07  0.222064  0.108128  0.179200  0.149451  0.177188  0.169813   
2012-09-10  0.207074  0.121212  0.148267  0.136484  0.151937  0.161554   
2012-09-11  0.228135  0.130327  0.176800  0.160440  0.177188  0.181023   
2012-09-12  0.227756  0.111357  0.173600  0.148352  0.167432  0.169027   
2012-09-13  0.242746  0.118667  0.160000  0.154066  0.168006  0.179646   
2012-09-14  0.226997  0.126351  0.176800  0.156484  0.180057  0.178073   
2012-09-17  0.222064  0.117137  0.184000  0.156484  0.179771  0.174336   
2012-09-18  0.232120  0.110990  0.173333  0.152747  0.176040  0.177

Let's change the scaler of our data, due to beforementioned facts.


In [27]:
from ipynb.fs.defs.GDL_data_prep import unit_centered_scaler

unit_stocknet_diff, inverse_unit_centered, invert_values = unit_centered_scaler(
                    compute_diff_ts(stocknet_df)
                )
unit_stocknet_diff.dropna(axis=1,inplace=True)
unit_stocknet_diff.head()

XOM                                                    \
                Open      High       Low     Close Adj Close    Volume   
Date                                                                     
2012-09-05  0.000013  0.000013  0.000036  0.000014  0.000012  0.014761   
2012-09-06  0.000025  0.000107  0.000053  0.000111  0.000095  0.022935   
2012-09-07  0.000093  0.000050  0.000095  0.000062  0.000053  0.022804   
2012-09-10  0.000037  0.000000  0.000009 -0.000030 -0.000025  0.015691   
2012-09-11 -0.000005 -0.000003  0.000010  0.000009  0.000008  0.013646   

                 AEP                                    ...       NGG  \
                Open      High           Low     Close  ...       Low   
Date                                                    ...             
2012-09-05  0.000024  0.000007  6.044507e-06 -0.000013  ...  0.000032   
2012-09-06 -0.000013  0.000015  7.388127e-06  0.000038  ...  0.000037   
2012-09-07  0.000027  0.000006  2.283582e-05 -0.000004  ... -0.000007   
2012-09-10  0.000009  0.000001  6.717761e-07 -0.000005  ... -0.000058   
2012-09-11 -0.000011 -0.000009  0.000000e+00  0.000003  ... -0.000016   

                                                UN                      \
               Close Adj Close    Volume      Open      High       Low   
Date                                                                     
2012-09-05  0.000021  0.000017  0.000413  0.000013  0.000010  0.000005   
2012-09-06  0.000043  0.000034  0.000492  0.000001  0.000038  0.000022   
2012-09-07 -0.000016 -0.000013  0.000299  0.000028 -0.000003  0.000019   
2012-09-10 -0.000074 -0.000058  0.000570 -0.000050 -0.000042 -0.000040   
2012-09-11  0.000007  0.000006  0.000758  0.000022  0.000031  0.000019   

                                          
               Close Adj Close    Volume  
Date                                      
2012-09-05  0.000001  0.000001  0.001771  
2012-09-06  0.000050  0.000042  0.003312  
2012-09-07 -0.000010 -0.000009  0.004299  
2012-09-10 -0.000038 -0.000032  0.003247  
2012-09-11  0.000028  0.000024  0.004917  

[5 rows x 486 columns]

Let's train the normalized data then. We will use the same model, just the data will be different.

In [28]:
tsl_unit_stock_df = SpatioTemporalDataset(target=unit_stocknet_diff,
                                     connectivity=edge_index_sector,
                                      horizon=5,
                                      window=12,
                                      stride=1)

stocknet_unit_datamodule = SpatioTemporalDataModule(
    dataset=tsl_unit_stock_df,
    splitter=stocknet_splitter,
    batch_size=BATCH_SIZE,
)

stocknet_unit_datamodule.setup()
print(stocknet_unit_datamodule)

SpatioTemporalDataModule(train_len=994, val_len=99, test_len=124, scalers=[], batch_size=64)


In [29]:
hidden_size = 16   #@param
rnn_layers = 1     #@param
gnn_kernel = 2     #@param

input_size = tsl_unit_stock_df.n_channels   # 6 channel
n_nodes = tsl_unit_stock_df.n_nodes         # 81 nodes
horizon = tsl_unit_stock_df.horizon         # 5 time steps

stocknet_unit_ttsm = TimeThenSpaceModel(input_size=input_size,
                           n_nodes=n_nodes,
                           horizon=horizon,
                           hidden_size=hidden_size,
                           rnn_layers=rnn_layers,
                           gnn_kernel=gnn_kernel)
print(stocknet_unit_ttsm)

TimeThenSpaceModel(
  (encoder): Linear(in_features=6, out_features=16, bias=True)
  (node_embeddings): NodeEmbedding(n_nodes=81, embedding_size=16)
  (time_nn): RNN(
    (rnn): GRU(16, 16)
  )
  (space_nn): DiffConv(16, 16)
  (decoder): Linear(in_features=16, out_features=30, bias=True)
  (rearrange): Rearrange('b n (t f) -> b t n f', t=5)
)


In [30]:

# setup predictor
stocknet_unit_ttsm_predictor = Predictor(
    model=stocknet_unit_ttsm,                   # our initialized model
    optim_class=torch.optim.Adam,  # specify optimizer to be used...
    optim_kwargs={'lr': 0.001},    # ...and parameters for its initialization
    loss_fn=stocknet_loss_fn,               # which loss function to be used
    metrics=stocknet_metrics                # metrics to be logged during train/val/test
)

MODEL_NAME = "stocknet-sector-unit-norm-window-12-forecast-5"
VERSION = 0
MODEL_PATH = SAVE_PATH / MODEL_NAME
stocknet_unit_logger = TensorBoardLogger(save_dir=SAVE_PATH, name=MODEL_NAME, version=VERSION)

Getting our logger ready

In [31]:
%load_ext tensorboard
%tensorboard --logdir $SAVE_PATH


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 13524), started 0:00:05 ago. (Use '!kill 13524' to kill it.)

In [32]:
stocknet_unit_checkpoint_callback = ModelCheckpoint(
    dirpath=MODEL_PATH,
    save_top_k=1,
    monitor='val_mae',
    mode='min',
    
)

MODEL_PATH.mkdir(exist_ok=True)

stocknet_unit_ttsm_trainer = pl.Trainer(max_epochs=230,
                     logger=stocknet_unit_logger,
                     limit_train_batches=2000,  # end an epoch after 100 updates
                     callbacks=[stocknet_unit_checkpoint_callback])
checkpoint_paths = [checkpoint for checkpoint in MODEL_PATH.iterdir() if checkpoint.suffix == ".ckpt"]
current_ckpt_path = checkpoint_paths[-1] if checkpoint_paths else None

stocknet_unit_ttsm_trainer.fit(stocknet_unit_ttsm_predictor, datamodule=stocknet_unit_datamodule, ckpt_path=current_ckpt_path)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/parf/.cache/pypoetry/virtualenvs/gnn-usi-24-tWyMBYAO-py3.11/lib/python3.11/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:653: Checkpoint directory /home/parf/projects/gnn-usi-24/experiment_data/stocknet-sector-unit-norm-window-12-forecast-5 exists and is not empty.
Restoring states from the checkpoint path at experiment_data/stocknet-sector-unit-norm-window-12-forecast-5/epoch=216-step=3255.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type               | Params
-----------------------------------------------------
0 | loss_fn       | MaskedMAE          | 0     
1 | train_metrics | MetricCollection   | 0     
2 | val_metrics   | MetricCollection   | 0     
3 | test_metrics  | MetricCollection   | 0     
4 | model         | TimeThenSpaceModel | 4.8 K 
------------------------------------------

Sanity Checking: |                                                                                                                          | 0/? [00:00<?, ?it/s]

/home/parf/.cache/pypoetry/virtualenvs/gnn-usi-24-tWyMBYAO-py3.11/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Sanity Checking DataLoader 0:   0%|                                                                                                         | 0/2 [00:00<?, ?it/s]

Only args ['edge_weight', 'x', 'edge_index'] are forwarded to the model (TimeThenSpaceModel). 


/home/parf/.cache/pypoetry/virtualenvs/gnn-usi-24-tWyMBYAO-py3.11/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.
/home/parf/.cache/pypoetry/virtualenvs/gnn-usi-24-tWyMBYAO-py3.11/lib/python3.11/site-packages/pytorch_lightning/loops/fit_loop.py:298: The number of training batches (15) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Epoch 217: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:00<00:00, 44.20it/s, v_num=0]
Validation: |                                                                                                                               | 0/? [00:00<?, ?it/s]
Epoch 218: 100%|███████████████████████████████████| 15/15 [00:00<00:00, 45.21it/s, v_num=0, val_mae=0.00188, val_mape=115.0, train_mae=0.00189, train_mape=97.80]
Validation: |                                                                                                                               | 0/? [00:00<?, ?it/s]
Epoch 219: 100%|███████████████████████████████████| 15/15 [00:00<00:00, 58.58it/s, v_num=0, val_mae=0.00194, val_mape=131.0, train_mae=0.00198, train_mape=126.0]
Validation: |                                                                                                                               | 0/? [00:00<?, ?it/s]
Epoch 220: 100%|██████

`Trainer.fit` stopped: `max_epochs=230` reached.


Epoch 229: 100%|███████████████████████████████████| 15/15 [00:00<00:00, 33.35it/s, v_num=0, val_mae=0.00193, val_mape=133.0, train_mae=0.00203, train_mape=146.0]


Let's design, as always, another big experiment.

In [33]:

PARAM_COMBINATIONS = [(10, 16, 2, 2), 
                     (100, 32, 4, 4)]


def conduct_stocknet_sector_unit_norm(window, hidden_size, rnn_layers, gnn_kernel):
    tsl_unit_stock_df = SpatioTemporalDataset(target=unit_stocknet_diff,
                                     connectivity=edge_index_sector,
                                      horizon=5,
                                      window=window,
                                      stride=1)

    stocknet_splitter = TemporalSplitter(val_len=VAL_LEN, test_len=TEST_LEN)


    stocknet_unit_datamodule = SpatioTemporalDataModule(
        dataset=tsl_unit_stock_df,
        splitter=stocknet_splitter,
        batch_size=BATCH_SIZE,
    )
    
    stocknet_unit_datamodule.setup()
    
    input_size = tsl_unit_stock_df.n_channels   # 6 channel
    n_nodes = tsl_unit_stock_df.n_nodes         # 81 nodes
    horizon = tsl_unit_stock_df.horizon         # 5 time steps
    
    stocknet_unit_ttsm = TimeThenSpaceModel(input_size=input_size,
                               n_nodes=n_nodes,
                               horizon=horizon,
                               hidden_size=hidden_size,
                               rnn_layers=rnn_layers,
                               gnn_kernel=gnn_kernel)

    stocknet_unit_ttsm_predictor = Predictor(
        model=stocknet_unit_ttsm,                   
        optim_class=torch.optim.Adam,  # specify optimizer to be used...
        optim_kwargs={'lr': 0.001},    # ...and parameters for its initialization
        loss_fn=stocknet_loss_fn,               # which loss function to be used
        metrics=stocknet_metrics                # metrics to be logged during train/val/test
    )
    
    MODEL_NAME = f"stocknet-sector-unit-norm-w-{window}-h-{hidden_size}-gnn-{gnn_kernel}-rnn-{rnn_layers}"
    VERSION = 0
    MODEL_PATH = SAVE_PATH / MODEL_NAME
    stocknet_unit_logger = TensorBoardLogger(save_dir=SAVE_PATH, name=MODEL_NAME, version=VERSION)


    stocknet_unit_checkpoint_callback = ModelCheckpoint(
    dirpath=MODEL_PATH,
    save_top_k=1,
    monitor='val_mae',
    mode='min',
    save_last=True

    )

    MODEL_PATH.mkdir(exist_ok=True)
    
    stocknet_unit_ttsm_trainer = pl.Trainer(max_epochs=EPOCH_COUNT,
                         logger=stocknet_unit_logger,
                         limit_train_batches=2000,  # end an epoch after 100 updates
                         callbacks=[stocknet_unit_checkpoint_callback])
    checkpoint_paths = [checkpoint for checkpoint in MODEL_PATH.iterdir() if checkpoint.suffix == ".ckpt"]
    current_ckpt_path = checkpoint_paths[-1] if checkpoint_paths else None
    
    stocknet_unit_ttsm_trainer.fit(stocknet_unit_ttsm_predictor, datamodule=stocknet_unit_datamodule, ckpt_path=current_ckpt_path)


if IS_TRAIN:
    for parameters_combination in PARAM_COMBINATIONS:
        conduct_stocknet_sector_unit_norm(*parameters_combination)

We are going to design a dataset for another task, namely an adjusted closing price trend prediction. For now we will use a converter, but it is a horrible thing to do :P

In [34]:
def convert_target_to_binary(data):
    adjusted_close = data.y[...,-2]
    adjusted_close_rises = (adjusted_close > 0)
    data.y = adjusted_close_rises.to(torch.float)
    return data


Let's then define our new dataset.

In [35]:
binary_classification_stocknet_df = SpatioTemporalDataset(target=unit_stocknet_diff, transform=convert_target_to_binary,
                                    connectivity=edge_index_sector,
                                      horizon=1,
                                      window=12,
                                      stride=1)
binary_classification_stocknet_df[0]

Data(
  input=(x=[t=12, n=81, f=6], edge_index=[2, e=751], edge_weight=[e=751]),
  target=(y=[t=1, n=81]),
  has_mask=False
)

Repeating the dm setup!

In [36]:

stocknet_binary_unit_datamodule = SpatioTemporalDataModule(
    dataset=binary_classification_stocknet_df,
    splitter=stocknet_splitter,
    batch_size=BATCH_SIZE,
)

stocknet_binary_unit_datamodule.setup()
print(stocknet_binary_unit_datamodule)

SpatioTemporalDataModule(train_len=994, val_len=99, test_len=124, scalers=[], batch_size=64)


Also we need to adjust our model a bit. Changing a way how we define it, mainly in the output though.

In [37]:
import torch.nn as nn

from tsl.nn.blocks.encoders import RNN
from tsl.nn.layers import NodeEmbedding, DiffConv
from einops.layers.torch import Rearrange  # reshape data with Einstein notation


class BinaryTimeThenSpaceModel(nn.Module):
    def __init__(self, input_size: int, n_nodes: int, horizon: int,
                 hidden_size: int = 32,
                 rnn_layers: int = 1,
                 gnn_kernel: int = 2):
        super(BinaryTimeThenSpaceModel, self).__init__()

        self.encoder = nn.Linear(input_size, hidden_size)

        self.node_embeddings = NodeEmbedding(n_nodes, hidden_size)

        self.time_nn = RNN(input_size=hidden_size,
                           hidden_size=hidden_size,
                           n_layers=rnn_layers,
                           cell='gru',
                           return_only_last_state=True)

        self.space_nn = DiffConv(in_channels=hidden_size,
                                 out_channels=hidden_size,
                                 k=gnn_kernel)

        self.decoder = nn.Linear(hidden_size, 1 * horizon)
        self.rearrange = Rearrange('b n (t) -> b t n', t=horizon)

    def forward(self, x, edge_index, edge_weight):
        # x: [batch time nodes features]
        x_enc = self.encoder(x)  # linear encoder: x_enc = xΘ + b
        x_emb = x_enc + self.node_embeddings()  # add node-identifier embeddings
        h = self.time_nn(x_emb)  # temporal processing: x=[b t n h] -> h=[b n h]
        z = self.space_nn(h, edge_index, edge_weight)  # spatial processing
        x_out = self.decoder(z)  # linear decoder: z=[b n h] -> x_out=[b n t⋅f]
        x_horizon = self.rearrange(x_out) # x_out=[b n t⋅f] -> x_out=[b t n f]
        return x_horizon



Then, let's initialize our model.

In [38]:
hidden_size = 16   #@param
rnn_layers = 1     #@param
gnn_kernel = 2     #@param

input_size = binary_classification_stocknet_df.n_channels   # 6 channel
n_nodes = binary_classification_stocknet_df.n_nodes         # 81 nodes
horizon = binary_classification_stocknet_df.horizon         # 5 time steps

binary_stocknet_ttsm = BinaryTimeThenSpaceModel(input_size=input_size,
                           n_nodes=n_nodes,
                           horizon=horizon,
                           hidden_size=hidden_size,
                           rnn_layers=rnn_layers,
                           gnn_kernel=gnn_kernel)
print(binary_stocknet_ttsm)

BinaryTimeThenSpaceModel(
  (encoder): Linear(in_features=6, out_features=16, bias=True)
  (node_embeddings): NodeEmbedding(n_nodes=81, embedding_size=16)
  (time_nn): RNN(
    (rnn): GRU(16, 16)
  )
  (space_nn): DiffConv(16, 16)
  (decoder): Linear(in_features=16, out_features=1, bias=True)
  (rearrange): Rearrange('b n (t) -> b t n', t=1)
)


We also need to redefine our predictor! 

In [39]:
import torch.nn.functional
import torchmetrics.functional.classification
import tsl.metrics.torch


class MaskedBCEWithLogits(tsl.metrics.torch.MaskedMetric):
    """
    """

    is_differentiable: bool = False
    higher_is_better: bool = False
    full_state_update: bool = False

    def __init__(self, mask_nans=False, mask_inf=False, at=None, **kwargs):
        super(MaskedBCEWithLogits, self).__init__(
            metric_fn=torch.nn.functional.binary_cross_entropy_with_logits,
            mask_nans=mask_nans,
            mask_inf=mask_inf,
            metric_fn_kwargs={'reduction': 'none'},
            at=at,
            **kwargs,
        )



class MaskedMCC(tsl.metrics.torch.MaskedMetric):
    """
    """

    is_differentiable: bool = False
    higher_is_better: bool = False
    full_state_update: bool = False

    def __init__(self, mask_nans=False, mask_inf=False, at=None, **kwargs):
        super(MaskedMCC, self).__init__(
            metric_fn=torchmetrics.functional.classification.binary_matthews_corrcoef,
            mask_nans=mask_nans,
            mask_inf=mask_inf,
            at=at,
            **kwargs,
        )


class MaskedAccuracy(tsl.metrics.torch.MaskedMetric):
    """
    """

    is_differentiable: bool = False
    higher_is_better: bool = False
    full_state_update: bool = False

    def __init__(self, mask_nans=False, mask_inf=False, at=None, **kwargs):
        super(MaskedAccuracy, self).__init__(
            metric_fn=torchmetrics.functional.classification.binary_accuracy,
            mask_nans=mask_nans,
            mask_inf=mask_inf,
            at=at,
            **kwargs,
        )




Then, let's redesign our support things, like loss and stuff.

In [40]:
stocknet_binary_loss_fn = MaskedBCEWithLogits()

stocknet_binary_metrics = {'mcc': MaskedMCC(), 'acc': MaskedAccuracy()}

# setup predictor
stocknet_binary_ttsm_predictor = Predictor(
    model=binary_stocknet_ttsm,                   # our initialized model
    optim_class=torch.optim.Adam,  # specify optimizer to be used...
    optim_kwargs={'lr': 0.001},    # ...and parameters for its initialization
    loss_fn=stocknet_binary_loss_fn,               # which loss function to be used
    metrics=stocknet_binary_metrics                # metrics to be logged during train/val/test
)

Let's initialize our logger then.

In [41]:
MODEL_NAME = "stocknet-sector-unit-classification-window-12-forecast-1"
VERSION = 0
MODEL_PATH = SAVE_PATH / MODEL_NAME

MODEL_PATH.mkdir(exist_ok=True)
stocknet_binary_logger = TensorBoardLogger(save_dir=SAVE_PATH, name=MODEL_NAME, version=VERSION)

Getting a tensorboard ready!

In [42]:
%load_ext tensorboard
%tensorboard --logdir $SAVE_PATH


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 13524), started 0:00:10 ago. (Use '!kill 13524' to kill it.)

Let's start a training process.

In [43]:

stocknet_binary_checkpoint_callback = ModelCheckpoint(
    dirpath=MODEL_PATH,
    save_top_k=1,
    monitor='val_acc',
    mode='min',
)

stocknet_binary_ttsm_trainer = pl.Trainer(max_epochs=10,
                     logger=stocknet_binary_logger,
                     limit_train_batches=2000,  # end an epoch after 100 updates
                     callbacks=[stocknet_binary_checkpoint_callback])
checkpoint_paths = [checkpoint for checkpoint in MODEL_PATH.iterdir() if checkpoint.suffix == ".ckpt"]
current_checkpoint_used = checkpoint_paths[0] if checkpoint_paths else None
stocknet_binary_ttsm_trainer.fit(stocknet_binary_ttsm_predictor, datamodule=stocknet_binary_unit_datamodule, ckpt_path=current_checkpoint_used)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/parf/.cache/pypoetry/virtualenvs/gnn-usi-24-tWyMBYAO-py3.11/lib/python3.11/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:653: Checkpoint directory /home/parf/projects/gnn-usi-24/experiment_data/stocknet-sector-unit-classification-window-12-forecast-1 exists and is not empty.
Restoring states from the checkpoint path at experiment_data/stocknet-sector-unit-classification-window-12-forecast-1/epoch=0-step=15.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type                     | Params
-----------------------------------------------------------
0 | loss_fn       | MaskedBCEWithLogits      | 0     
1 | train_metrics | MetricCollection         | 0     
2 | val_metrics   | MetricCollection         | 0     
3 | test_metrics  | MetricCollection         | 0     
4 | model         | BinaryTimeThenSpac

Sanity Checking: |                                                                                                                          | 0/? [00:00<?, ?it/s]

/home/parf/.cache/pypoetry/virtualenvs/gnn-usi-24-tWyMBYAO-py3.11/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Sanity Checking DataLoader 0:   0%|                                                                                                         | 0/2 [00:00<?, ?it/s]

Only args ['edge_weight', 'x', 'edge_index'] are forwarded to the model (BinaryTimeThenSpaceModel). 


/home/parf/.cache/pypoetry/virtualenvs/gnn-usi-24-tWyMBYAO-py3.11/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.
/home/parf/.cache/pypoetry/virtualenvs/gnn-usi-24-tWyMBYAO-py3.11/lib/python3.11/site-packages/pytorch_lightning/loops/fit_loop.py:298: The number of training batches (15) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Epoch 1: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:00<00:00, 17.02it/s, v_num=0]
Validation: |                                                                                                                               | 0/? [00:00<?, ?it/s]
Epoch 2: 100%|████████████████████████████████████| 15/15 [00:00<00:00, 28.30it/s, v_num=0, val_acc=0.519, val_mcc=-0.00692, train_acc=0.483, train_mcc=-0.000926]
Validation: |                                                                                                                               | 0/? [00:00<?, ?it/s]
Epoch 3: 100%|██████████████████████████████████████| 15/15 [00:00<00:00, 30.28it/s, v_num=0, val_acc=0.519, val_mcc=-0.00488, train_acc=0.516, train_mcc=0.00721]
Validation: |                                                                                                                               | 0/? [00:00<?, ?it/s]
Epoch 4: 100%|████████

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|███████████████████████████████████████| 15/15 [00:00<00:00, 31.93it/s, v_num=0, val_acc=0.518, val_mcc=-0.00515, train_acc=0.522, train_mcc=0.0212]


Let's see the results of it.

In [44]:
stocknet_binary_ttsm_predictor.load_model(stocknet_binary_checkpoint_callback.best_model_path)
stocknet_binary_ttsm_predictor.freeze()

stocknet_binary_ttsm_trainer.test(stocknet_binary_ttsm_predictor, datamodule=stocknet_binary_unit_datamodule);

Predictor with already instantiated model is loading a state_dict from /home/parf/projects/gnn-usi-24/experiment_data/stocknet-sector-unit-classification-window-12-forecast-1/epoch=0-step=15.ckpt. Cannot  check if model hyperparameters are the same.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/parf/.cache/pypoetry/virtualenvs/gnn-usi-24-tWyMBYAO-py3.11/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 59.68it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc            0.4738618731498718
        test_loss           0.6922172904014587
        test_mcc           -0.001469177077524364
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


Let's design an experiment group again. 

In [45]:
PARAM_COMBINATIONS = [(10, 16, 2, 2), 
                     (100, 32, 4, 4)]


def conduct_sector_unit_classification(window, hidden_size, rnn_layers, gnn_kernel):
    binary_classification_stocknet_df = SpatioTemporalDataset(target=unit_stocknet_diff, transform=convert_target_to_binary,
                                    connectivity=edge_index_sector,
                                      horizon=1,
                                      window=window,
                                      stride=1)

    stocknet_splitter = TemporalSplitter(val_len=VAL_LEN, test_len=TEST_LEN)

    stocknet_binary_unit_datamodule = SpatioTemporalDataModule(
        dataset=binary_classification_stocknet_df,
        splitter=stocknet_splitter,
        batch_size=BATCH_SIZE,
    )
    
    stocknet_binary_unit_datamodule.setup()

    input_size = binary_classification_stocknet_df.n_channels   # 6 channel
    n_nodes = binary_classification_stocknet_df.n_nodes         # 81 nodes
    horizon = binary_classification_stocknet_df.horizon         # 5 time steps
    
    binary_stocknet_ttsm = BinaryTimeThenSpaceModel(input_size=input_size,
                               n_nodes=n_nodes,
                               horizon=horizon,
                               hidden_size=hidden_size,
                               rnn_layers=rnn_layers,
                               gnn_kernel=gnn_kernel)

    stocknet_binary_loss_fn = MaskedBCEWithLogits()

    stocknet_binary_metrics = {'mcc': MaskedMCC(), 'acc': MaskedAccuracy()}
    
    # setup predictor
    stocknet_binary_ttsm_predictor = Predictor(
        model=binary_stocknet_ttsm,                   # our initialized model
        optim_class=torch.optim.Adam,  # specify optimizer to be used...
        optim_kwargs={'lr': 0.001},    # ...and parameters for its initialization
        loss_fn=stocknet_binary_loss_fn,               # which loss function to be used
        metrics=stocknet_binary_metrics                # metrics to be logged during train/val/test
    )

    MODEL_NAME = f"stocknet-ttsm-classification-{window}-h-{hidden_size}-gnn-{gnn_kernel}-rnn-{rnn_layers}"
    VERSION = 0
    MODEL_PATH = SAVE_PATH / MODEL_NAME
    
    MODEL_PATH.mkdir(exist_ok=True)
    stocknet_binary_logger = TensorBoardLogger(save_dir=SAVE_PATH, name=MODEL_NAME, version=VERSION)

    
    stocknet_binary_checkpoint_callback = ModelCheckpoint(
        dirpath=MODEL_PATH,
        save_top_k=1,
        monitor='val_acc',
        mode='min',
        save_last=True

    )
    
    stocknet_binary_ttsm_trainer = pl.Trainer(max_epochs=EPOCH_COUNT,
                         logger=stocknet_binary_logger,
                         limit_train_batches=2000,  # end an epoch after 100 updates
                         callbacks=[stocknet_binary_checkpoint_callback])
    checkpoint_paths = [checkpoint for checkpoint in MODEL_PATH.iterdir() if checkpoint.suffix == ".ckpt"]
    current_checkpoint_used = checkpoint_paths[0] if checkpoint_paths else None
    stocknet_binary_ttsm_trainer.fit(stocknet_binary_ttsm_predictor, datamodule=stocknet_binary_unit_datamodule, ckpt_path=current_checkpoint_used)




if True:
    for parameters_combination in PARAM_COMBINATIONS:
        conduct_sector_unit_classification(*parameters_combination)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/parf/.cache/pypoetry/virtualenvs/gnn-usi-24-tWyMBYAO-py3.11/lib/python3.11/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:653: Checkpoint directory /home/parf/projects/gnn-usi-24/experiment_data/stocknet-ttsm-classification-10-h-16-gnn-2-rnn-2 exists and is not empty.
Restoring states from the checkpoint path at experiment_data/stocknet-ttsm-classification-10-h-16-gnn-2-rnn-2/epoch=1-step=30.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type                     | Params
-----------------------------------------------------------
0 | loss_fn       | MaskedBCEWithLogits      | 0     
1 | train_metrics | MetricCollection         | 0     
2 | val_metrics   | MetricCollection         | 0     
3 | test_metrics  | MetricCollection         | 0     
4 | model         | BinaryTimeThenSpaceModel | 6.0 K 


Sanity Checking: |                                                                                                                          | 0/? [00:00<?, ?it/s]

/home/parf/.cache/pypoetry/virtualenvs/gnn-usi-24-tWyMBYAO-py3.11/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Sanity Checking DataLoader 0:   0%|                                                                                                         | 0/2 [00:00<?, ?it/s]

Only args ['edge_weight', 'x', 'edge_index'] are forwarded to the model (BinaryTimeThenSpaceModel). 


...

Only args ['edge_weight', 'x', 'edge_index'] are forwarded to the model (BinaryTimeThenSpaceModel). 


/home/parf/.cache/pypoetry/virtualenvs/gnn-usi-24-tWyMBYAO-py3.11/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.
/home/parf/.cache/pypoetry/virtualenvs/gnn-usi-24-tWyMBYAO-py3.11/lib/python3.11/site-packages/pytorch_lightning/loops/fit_loop.py:298: The number of training batches (13) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Epoch 1: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 13/13 [00:02<00:00,  6.37it/s, v_num=0]
Validation: |                                                                                                                               | 0/? [00:00<?, ?it/s]
Epoch 1: 100%|███████████████████████████████████████| 13/13 [00:02<00:00,  6.27it/s, v_num=0, val_acc=0.546, val_mcc=0.0103, train_acc=0.488, train_mcc=0.000181]

`Trainer.fit` stopped: `max_epochs=2` reached.


Epoch 1: 100%|███████████████████████████████████████| 13/13 [00:02<00:00,  6.26it/s, v_num=0, val_acc=0.546, val_mcc=0.0103, train_acc=0.488, train_mcc=0.000181]


Let's design a new model for our purposes. It will be a Space then Time model. 

In [46]:
import torch.nn as nn
from tsl.nn.layers import GatedGraphNetwork


class BinarySpaceThenTimeModel(nn.Module):
    def __init__(self, input_size: int,
                 n_nodes: int, 
                 horizon: int,
                 window: int,
                 hidden_size: int = 32,
                 mp_layers: int = 2,
                 rnn_layers: int = 1):
        super(BinarySpaceThenTimeModel, self).__init__()

        self.encoder = nn.Linear(input_size, hidden_size)

        self.node_embeddings = NodeEmbedding(n_nodes, hidden_size)

        self.time_nn = RNN(input_size=hidden_size,
                           hidden_size=hidden_size,
                           n_layers=rnn_layers,
                           cell='gru',
                           return_only_last_state=True)
        self.rearrange_time_forward = Rearrange('b t n f -> t b n f', t=window)
        space_nns_timestep_layers = nn.ModuleList([GatedGraphNetwork(input_size=hidden_size,
                                 output_size=hidden_size) for window_idx in range(window)])

        self.space_nns_layers = nn.ModuleList([space_nns_timestep_layers for mp_layer_idx in range(mp_layers)])

        self.rearrange_time_second = Rearrange('t b n f -> b t n f', t=window)


        self.decoder = nn.Linear(hidden_size, 1 * horizon)
        self.rearrange = Rearrange('b n (t) -> b t n', t=horizon)

    def forward(self, x, edge_index, edge_weight):
        # x: [batch time nodes features]
        x_encoded = self.encoder(x)  
        x_embedding = x_encoded + self.node_embeddings() 
        x_timestepped = self.rearrange_time_forward(x_embedding)
        for mp_layers_group in self.space_nns_layers:
            message_passed_x = []
            for timestep_idx, x_timestep in enumerate(x_timestepped):
                message_passed = mp_layers_group[timestep_idx](x_timestep, edge_index)
                message_passed_x.append(message_passed)
            x_timestepped = torch.stack(message_passed_x)
        rearranged_message_passed = self.rearrange_time_second(x_timestepped)
        time_propagation = self.time_nn(rearranged_message_passed)  # temporal processing: x=[b t n h] -> h=[b n h]
        x_out = self.decoder(time_propagation)  # linear decoder: z=[b n h] -> x_out=[b n t⋅f]
        x_horizon = self.rearrange(x_out) # x_out=[b n t⋅f] -> x_out=[b t n f]
        return x_horizon


Let's reset our data module to be sure.

In [47]:

stocknet_binary_unit_datamodule = SpatioTemporalDataModule(
    dataset=binary_classification_stocknet_df,
    splitter=stocknet_splitter,
    batch_size=BATCH_SIZE,
)

stocknet_binary_unit_datamodule.setup()
print(stocknet_binary_unit_datamodule)

SpatioTemporalDataModule(train_len=994, val_len=99, test_len=124, scalers=[], batch_size=64)


Let's initialize our model then.

In [48]:
hidden_size = 16   #@param
rnn_layers = 1     #@param
mp_layers =2

window = binary_classification_stocknet_df.window
input_size = binary_classification_stocknet_df.n_channels   # 6 channel
n_nodes = binary_classification_stocknet_df.n_nodes         # 81 nodes
horizon = binary_classification_stocknet_df.horizon         # 5 time steps

binary_stocknet_sttm = BinarySpaceThenTimeModel(input_size=input_size,
                                                n_nodes=n_nodes,
                                                horizon=horizon,
                                                window = window,
                                                hidden_size=hidden_size,
                                                mp_layers = mp_layers,
                                                rnn_layers=rnn_layers)

binary_stocknet_sttm

BinarySpaceThenTimeModel(
  (encoder): Linear(in_features=6, out_features=16, bias=True)
  (node_embeddings): NodeEmbedding(n_nodes=81, embedding_size=16)
  (time_nn): RNN(
    (rnn): GRU(16, 16)
  )
  (rearrange_time_forward): Rearrange('b t n f -> t b n f', t=12)
  (space_nns_layers): ModuleList(
    (0-1): 2 x ModuleList(
      (0-11): 12 x GatedGraphNetwork(16, 16)
    )
  )
  (rearrange_time_second): Rearrange('t b n f -> b t n f', t=12)
  (decoder): Linear(in_features=16, out_features=1, bias=True)
  (rearrange): Rearrange('b n (t) -> b t n', t=1)
)

Making the new predictor!

In [49]:
stocknet_binary_loss_fn = MaskedBCEWithLogits()

stocknet_binary_metrics = {'mcc': MaskedMCC(), 'acc': MaskedAccuracy()}

# setup predictor
stocknet_binary_sttm_predictor = Predictor(
    model=binary_stocknet_sttm,                   # our initialized model
    optim_class=torch.optim.Adam,  # specify optimizer to be used...
    optim_kwargs={'lr': 0.001},    # ...and parameters for its initialization
    loss_fn=stocknet_binary_loss_fn,               # which loss function to be used
    metrics=stocknet_binary_metrics                # metrics to be logged during train/val/test
)

Making new logging possible

In [50]:
MODEL_NAME = "stocknet-sttm-unit-classification-window-12-forecast-1"
VERSION = 0
MODEL_PATH = SAVE_PATH / MODEL_NAME

MODEL_PATH.mkdir(exist_ok=True)
stocknet_sttm_binary_logger = TensorBoardLogger(save_dir=SAVE_PATH, name=MODEL_NAME, version=VERSION)

Getting the tb ready.

In [51]:
%load_ext tensorboard
%tensorboard --logdir $SAVE_PATH


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 13524), started 0:00:18 ago. (Use '!kill 13524' to kill it.)

And finally running the training process.

In [52]:

stocknet_binary_sttm_checkpoint_callback = ModelCheckpoint(
    dirpath=MODEL_PATH,
    save_top_k=1,
    monitor='val_acc',
    mode='min',
)

stocknet_binary_sttm_trainer = pl.Trainer(max_epochs=10,
                     logger=stocknet_sttm_binary_logger,
                     limit_train_batches=2000,  # end an epoch after 100 updates
                     callbacks=[stocknet_binary_sttm_checkpoint_callback])
checkpoint_paths = [checkpoint for checkpoint in MODEL_PATH.iterdir() if checkpoint.suffix == ".ckpt"]
current_checkpoint_used = checkpoint_paths[0] if checkpoint_paths else None
stocknet_binary_sttm_trainer.fit(stocknet_binary_sttm_predictor, datamodule=stocknet_binary_unit_datamodule, ckpt_path=current_checkpoint_used)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/parf/.cache/pypoetry/virtualenvs/gnn-usi-24-tWyMBYAO-py3.11/lib/python3.11/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:653: Checkpoint directory /home/parf/projects/gnn-usi-24/experiment_data/stocknet-sttm-unit-classification-window-12-forecast-1 exists and is not empty.
Restoring states from the checkpoint path at experiment_data/stocknet-sttm-unit-classification-window-12-forecast-1/epoch=2-step=45.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type                     | Params
-----------------------------------------------------------
0 | loss_fn       | MaskedBCEWithLogits      | 0     
1 | train_metrics | MetricCollection         | 0     
2 | val_metrics   | MetricCollection         | 0     
3 | test_metrics  | MetricCollection         | 0     
4 | model         | BinarySpaceThenTimeMod

Sanity Checking DataLoader 0:   0%|                                                                                                         | 0/2 [00:00<?, ?it/s]

Only args ['edge_weight', 'x', 'edge_index'] are forwarded to the model (BinarySpaceThenTimeModel). 


/home/parf/.cache/pypoetry/virtualenvs/gnn-usi-24-tWyMBYAO-py3.11/lib/python3.11/site-packages/pytorch_lightning/loops/fit_loop.py:298: The number of training batches (15) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Epoch 3: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:02<00:00,  6.03it/s, v_num=0]
Validation: |                                                                                                                               | 0/? [00:00<?, ?it/s]
Epoch 4: 100%|████████████████████████████████████| 15/15 [00:02<00:00,  6.09it/s, v_num=0, val_acc=0.489, val_mcc=-0.00801, train_acc=0.485, train_mcc=-0.000854]
Validation: |                                                                                                                               | 0/? [00:00<?, ?it/s]
Epoch 5: 100%|███████████████████████████████████████| 15/15 [00:02<00:00,  7.19it/s, v_num=0, val_acc=0.520, val_mcc=0.00622, train_acc=0.525, train_mcc=0.00359]
Validation: |                                                                                                                               | 0/? [00:00<?, ?it/s]
Epoch 6: 100%|████████

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|█████████████████████████████████████████| 15/15 [00:02<00:00,  6.78it/s, v_num=0, val_acc=0.521, val_mcc=0.0102, train_acc=0.518, train_mcc=0.0189]


Let's design an experiment then.

In [53]:

PARAM_COMBINATIONS = [(10, 16, 1,1,),
                      (20, 32, 2,4,)]


def conduct_stocknet_sttm_unit_classification(window, hidden_size, rnn_layers, mp_layers,):

    binary_classification_stocknet_df = SpatioTemporalDataset(target=unit_stocknet_diff,
                                                              transform=convert_target_to_binary,
                                    connectivity=edge_index_sector,
                                      horizon=1,
                                      window=window,
                                      stride=1)

    stocknet_splitter = TemporalSplitter(val_len=VAL_LEN, test_len=TEST_LEN)

    
    stocknet_binary_unit_datamodule = SpatioTemporalDataModule(
        dataset=binary_classification_stocknet_df,
        splitter=stocknet_splitter,
        batch_size=BATCH_SIZE,
    )
    
    stocknet_binary_unit_datamodule.setup()

    window = binary_classification_stocknet_df.window
    input_size = binary_classification_stocknet_df.n_channels   # 6 channel
    n_nodes = binary_classification_stocknet_df.n_nodes         # 81 nodes
    horizon = binary_classification_stocknet_df.horizon         # 5 time steps
    
    binary_stocknet_sttm = BinarySpaceThenTimeModel(input_size=input_size,
                                                    n_nodes=n_nodes,
                                                    horizon=horizon,
                                                    window = window,
                                                    hidden_size=hidden_size,
                                                    mp_layers = mp_layers,
                                                    rnn_layers=rnn_layers)
    
    
    stocknet_binary_loss_fn = MaskedBCEWithLogits()

    stocknet_binary_metrics = {'mcc': MaskedMCC(), 'acc': MaskedAccuracy()}
    
    # setup predictor
    stocknet_binary_sttm_predictor = Predictor(
        model=binary_stocknet_sttm,                   # our initialized model
        optim_class=torch.optim.Adam,  # specify optimizer to be used...
        optim_kwargs={'lr': 0.001},    # ...and parameters for its initialization
        loss_fn=stocknet_binary_loss_fn,               # which loss function to be used
        metrics=stocknet_binary_metrics                # metrics to be logged during train/val/test
    )

    MODEL_NAME = f"stocknet-sttm-unit-classification-w-{window}-h-{hidden_size}-mp-{mp_layers}-rnn-{rnn_layers}"
    VERSION = 0
    MODEL_PATH = SAVE_PATH / MODEL_NAME
    
    MODEL_PATH.mkdir(exist_ok=True)
    stocknet_sttm_binary_logger = TensorBoardLogger(save_dir=SAVE_PATH, name=MODEL_NAME, version=VERSION)
    
    stocknet_binary_sttm_checkpoint_callback = ModelCheckpoint(
        dirpath=MODEL_PATH,
        save_top_k=1,
        monitor='val_acc',
        mode='min',
        save_last=True

    )
    
    stocknet_binary_sttm_trainer = pl.Trainer(max_epochs=EPOCH_COUNT,
                         logger=stocknet_sttm_binary_logger,
                         limit_train_batches=2000,  # end an epoch after 100 updates
                         callbacks=[stocknet_binary_sttm_checkpoint_callback])
    checkpoint_paths = [checkpoint for checkpoint in MODEL_PATH.iterdir() if checkpoint.suffix == ".ckpt"]
    current_checkpoint_used = checkpoint_paths[0] if checkpoint_paths else None
    stocknet_binary_sttm_trainer.fit(stocknet_binary_sttm_predictor, datamodule=stocknet_binary_unit_datamodule, ckpt_path=current_checkpoint_used)

if IS_TRAIN:
    for param_combination in PARAM_COMBINATIONS:
        conduct_stocknet_sttm_unit_classification(*param_combination)

We also need to redefine our function converter for targets.

In [54]:
def convert_target_to_binary_from_abs(data):
    adjusted_close = data.y[...,-2] - data.x[..., -2]
    data.y = adjusted_close
    print(data.x)
    return data


Let's also try the previous model on another dataset probably, with another scaling startegy. To be sure.

In [55]:
from tsl.data.preprocessing import StandardScaler

scalers = {'target': StandardScaler()}

stocknet_abs_diff = stocknet_df.dropna(axis=1,inplace=False)


stocknet_binary_absolute_scaled_dataset = SpatioTemporalDataset(target=stocknet_abs_diff, 
                                                                   transform=convert_target_to_binary_from_abs,
                                                                   connectivity=edge_index_sector,
                                                                   horizon=1,
                                                                   window=12,
                                                                   stride=1)



stocknet_binary_absolute_scaled_datamodule = SpatioTemporalDataModule(
    dataset=stocknet_binary_absolute_scaled_dataset,
    splitter=stocknet_splitter,
    scalers=scalers,
    batch_size=BATCH_SIZE,
)
stocknet_binary_absolute_scaled_datamodule.setup()
next(iter(stocknet_binary_absolute_scaled_datamodule.train_dataloader())).x.max()

tensor([[[[-3.0589e-01, -3.3948e-01, -2.5869e-01, -3.0411e-01, -9.7654e-01,
           -8.6480e-01],
          [-1.5745e+00, -1.6013e+00, -1.6379e+00, -1.6444e+00, -1.5711e+00,
            8.9605e-03],
          [-2.0307e+00, -2.0468e+00, -2.0362e+00, -2.0444e+00, -1.9598e+00,
            4.4000e-01],
          ...,
          [-2.0066e+00, -2.0977e+00, -2.0247e+00, -2.0812e+00, -1.9317e+00,
           -7.1797e-01],
          [-1.8504e+00, -1.8819e+00, -1.8861e+00, -1.8887e+00, -1.8742e+00,
           -2.1894e-01],
          [-1.9438e+00, -1.9523e+00, -1.9055e+00, -1.9621e+00, -1.8274e+00,
            5.4983e-01]],

         [[-3.8088e-01, -2.9567e-01, -3.3158e-01, -4.2691e-01, -1.1040e+00,
           -1.4046e-01],
          [-1.6616e+00, -1.6310e+00, -1.6310e+00, -1.6349e+00, -1.5644e+00,
           -7.2057e-02],
          [-2.0515e+00, -2.0422e+00, -2.0292e+00, -2.0575e+00, -1.9700e+00,
           -5.5464e-02],
          ...,
          [-2.1185e+00, -1.9825e+00, -2.0290e+00, -2.0471e+

tensor(28.2127)